# Шахматный движок на PyTorch

## Импорты

In [1]:
import os
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from chess import pgn
from tqdm import tqdm

# Подготовка данных

## Загрузка данных

In [2]:
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir("../../data/pgn") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(len(files), 28)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"../../data/pgn/{file}"))
    if i >= LIMIT_OF_FILES:
        break
    i += 1

 34%|███▍      | 27/79 [03:08<06:03,  6.99s/it]


In [3]:
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 41570


## Преобразование данных в тензоры

In [4]:
from auxiliary_func import create_input_for_nn, encode_moves

In [5]:
X, y = create_input_for_nn(games)

print(f"NUMBER OF SAMPLES: {len(y)}")

NUMBER OF SAMPLES: 3332761


In [6]:
X = X[0:2500000]
y = y[0:2500000]

In [7]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [8]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Создание набора данных

In [9]:
from dataset import ChessDataset
from model import ChessModel

In [10]:
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Проверка доступности CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Инициализация модели
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cuda


# Обучение

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        # В моём случае вычисления будут производиться на видеокарте
        # Если CUDA не установлена, вычисления будут производиться на центральном процессоре
        optimizer.zero_grad()

        outputs = model(inputs)  # Необработанные логиты

        # Вычисление издержек
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Обрезка градиента
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f'Epoch {epoch}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')

100%|██████████| 39063/39063 [04:46<00:00, 136.48it/s]


Epoch 0/100, Loss: 3.5637, Time: 4m46s


100%|██████████| 39063/39063 [05:37<00:00, 115.82it/s]


Epoch 1/100, Loss: 2.6847, Time: 5m37s


100%|██████████| 39063/39063 [04:36<00:00, 141.39it/s]


...

100%|██████████| 39063/39063 [04:43<00:00, 137.75it/s]


Epoch 71/100, Loss: 1.4949, Time: 4m43s


100%|██████████| 39063/39063 [04:26<00:00, 146.58it/s]


Epoch 72/100, Loss: 1.4920, Time: 4m26s


 27%|██▋       | 10501/39063 [01:11<03:16, 145.54it/s]

# Сохранение модели и маппинга

In [ ]:
torch.save(model.state_dict(), "../../models/TORCH_100EPOCHS.pth")

In [ ]:
import pickle

with open("../../models/heavy_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)